Install libraries

In [ ]:
#Need to install if not installed yet
!pip install matplotlib
!pip install scipy
!pip install sklearn
!pip install pandas
!pip install seaborn
!pip install haversine
!pip install -U wxPython 
!pip install google
!pip install google-api-core
!pip install --upgrade google-api-python-client
!pip install google-cloud
!pip install google-cloud-vision
!pip install google.cloud.bigquery
!pip install google.cloud.storage
!pip install google-auth-oauthlib


Import Libraries

In [1]:
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import uniform
from scipy.stats import norm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
import csv
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import seaborn as sn
import requests
import json
import haversine as hs
import wx
import webbrowser

from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import DQ2# Own defined
#hola

Import Setup file format

In [2]:
## Weights
#mu                =0.9
#Accuracy          =0.20
#Precision         =0.07
#Confidence        =0.16
#Completeness      =0.10
#Timeliness        =0.12
#Data_Volume       =0.16
#Data_Redundancy   =0.02
#Concordance       =0.16
#
#Utility           =0.12
#Accessibility     =0.16
#Interpretability  =0.28
#Reputation        =0.12
#Artificiality     =0.20
#Access_Security   =0.12


mu              = 1.0
Waccuracy        = 0.3506311521
Wprecision       = 0.09875987987
Wconfidence      = 0.1880884436
Wcompleteness    = 0.148093351
Wdata_Redundancy = 0.03756434625
Wconcordance     = 0.1768628272
pcmWeights = [Waccuracy,Wprecision,Wconfidence,Wcompleteness,Wdata_Redundancy,Wconcordance]

#Period
start_time ="2019-12-20 00:00:00"
end_time   ="2019-12-29 23:59:00"

#Variable Inicialization
p=99 # P for the CI used in the confidence calculation


The steps should be like:

0. Clean the whole dataset (the variables of interest):DONE
1. For each citizen science (CC) node, get the groups (HOURLY GROUPS).
2. For each group (hour in a CC node data), calculate the Dimension's DQ. (The functions should be applied to each group instead)
3. Save the result file in the form: Node, Group (hour), DQ_1, DQ_2, DQ3, ... , DQIndex  (This is new)
4. Average the previous result over the whole time to get Node, DQ_1_time_mean, DQ_2_time_mean, DQ3_ave, ... , DQIndex_time_mean (This it what we have currently)
5. Average the previous result over all the nodes to get DQ_1_node_mean, DQ_2_node_mean, DQ3_node_mean, ... , DQIndex_node_mean (This is new)
6. Export 3, 4 and 5 to a Google Sheets page each one.

Read Data

In [3]:
try:
    
#Read Data from February
    header_CC=["codigoSerial", "fecha", "hora", "fechaHora", "temperatura", "humedad_relativa", "pm1_df", "pm10_df", "pm25_df", "pm1_nova", "pm10_nova", "pm25_nova", "calidad_temperatura", "calidad_humedad_relativa", "calidad_pm1_df", "calidad_pm10_df", "calidad_pm25_df", "calidad_pm1_nova", "calidad_pm10_nova", "calidad_pm25_nova"]
    datatypes_CC={"codigoSerial":np.uint16, "temperatura":np.float16, "humedad_relativa":np.float16, "pm1_df":np.float32, "pm10_df":np.float32, "pm25_df":np.float32, "pm1_nova":np.float32, "pm10_nova":np.float32, "pm25_nova":np.float32}
    path_for_CC_data=DQ2.get_path('*.csv',"Select Citizen Scientist *.csv file")
    df_CC = pd.read_csv(path_for_CC_data, header=None, names=header_CC, usecols=header_CC , dtype=datatypes_CC,parse_dates=["fecha","hora","fechaHora"])
    df_CC.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    print("Source path for Citizen Science nodes data: ",path_for_CC_data)
    
    #Data includes January, February and March
    header_SS=["Fecha_Hora","codigoSerial","pm25","calidad_pm25","pm10","calidad_pm10"]
    datatypes_SS={"codigoSerial":np.uint16,"pm25":np.float32,"pm10":np.float32}
    path_for_SS_data=DQ2.get_path('*.csv',"Select SIATA Stations *.csv file")
    df_SS = pd.read_csv(path_for_SS_data, header=None,names=header_SS, usecols=header_SS , dtype=datatypes_SS,parse_dates=["Fecha_Hora"])
    df_SS.sort_values(by=['codigoSerial','Fecha_Hora'],ignore_index=True)
    print("Source path for Siata Stations data: ",path_for_SS_data)
    
    
    datatypesDistances={"codigoSerial_CC":np.uint16,"codigoSerial_ES":np.uint16,"Distancia_a_ES":np.float16,"codigoSerial_ES2":np.uint16}
    path_for_distance_files=DQ2.get_path('*.csv',"Select the Nodes to Siata Stations distances *.csv file")
    Distances = pd.read_csv(path_for_distance_files, header=0, dtype=datatypesDistances,index_col="codigoSerial_CC")
    print("The distance files was read")
    
        
except:
    print("An exception occurred, it is possible that wrong files were chosen, please run again")



#DATA CLEANING
CC, SS=DQ2.clean_sort_data(df_CC, df_SS)
del df_CC
del df_SS


Source path for Citizen Science nodes data:  C:\Users\julio\Documents\UDEA\Maestría\DQ in IOT\Datasets\Dataset for testing\CC_PM_December.csv
Source path for Siata Stations data:  C:\Users\julio\Documents\UDEA\Maestría\DQ in IOT\Datasets\Dataset for testing\SS_PM December 2019 till March 2020.csv
The distance files was read
Citizen Scientist:  [1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 90, 91, 92, 94, 95, 96, 97, 99, 100, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 149, 150, 151, 152, 153, 154, 156, 157, 158, 160, 161, 162, 165, 166, 167, 168, 169, 170, 172, 173, 174, 175, 176, 179,

Main Code with parallelization

In [4]:
import DQ2# Own defined
t0= time.time()
print("Start time: ", datetime.fromtimestamp(t0))


dim_time = pd.DataFrame(
        columns =["codigoSerial",
                  "fechaHora",
                  "precision_df_time",
                  "precision_nova_time",
                  "uncertainty_time",
                  "accuracy_df_time",
                  "accuracy_nova_time",
                  "completeness_df_time",
                  "completeness_nova_time",
                  "concordance_df_nova_time",
                  "concordance_df_siata_time",
                  "concordance_df_hum_time",
                  "concordance_df_temp_time",
                  "concordance_nova_siata_time",
                  "concordance_nova_hum_time",
                  "concordance_nova_temp_time",
                  "vm_df",
                  "vm_nova",
                  "v",
                  "duplicates_time",
                  
                  "confi_df_time",
                  "confi_nova_time"])

dim_node = pd.DataFrame(
        columns =["codigoSerial",
                  "precision_df_node",
                  "precision_nova_node",
                  "uncertainty_node",
                  "accuracy_df_node",
                  "accuracy_nova_node",
                  "completeness_df_node",
                  "completeness_nova_node",
                  "concordance_df_nova_node",
                  "concordance_df_siata_node",
                  "concordance_df_hum_node",
                  "concordance_df_temp_node",
                  "concordance_nova_siata_node",
                  "concordance_nova_hum_node",
                  "concordance_nova_temp_node",
                  
                  "duplicates_node",
                  
                  "confi_df_node",
                  "confi_nova_node",
                  "DQ_INDEX_NODE"])

dim_DQ = pd.DataFrame(
        columns =["precision_df_total",
                  "precision_nova_total",
                  "uncertainty_total",
                  "accuracy_df_total",
                  "accuracy_nova_total",
                  "completeness_df_total",
                  "completeness_nova_total",
                  "concordance_df_nova_total",
                  "concordance_df_siata_total",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_df_hum_total",
                  "concordance_df_temp_total",
                  "concordance_nova_siata_total",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_nova_hum_total",
                  "concordance_nova_temp_total",
                  
                  "duplicates_total",
                  
                  "confi_df_total",
                  "confi_nova_total",
                  "DQ_INDEX_TOTAL"])

if __name__ ==  '__main__':
    print("Number of avaliable CPUs: ",mp.cpu_count())
    pool=mp.Pool(processes = mp.cpu_count())
    arguments=[]
    #results=pool.map(DQ.eval_dq,[nodes for nodes in CC.keys()])
    results=pool.map(DQ2.eval_dq,([[nodes, CC, SS, Distances, start_time, end_time, p] for nodes in CC.keys()]))
    


    for i in range(0,len(results)):
        dim_time=dim_time.append(results[i][0], ignore_index = True)
        dim_node=dim_node.append(results[i][1], ignore_index = True)

        
    cols =[       "precision_df_node",
                  "precision_nova_node",
                  "uncertainty_node",
                  "accuracy_df_node",
                  "accuracy_nova_node",
                  "completeness_df_node",
                  "completeness_nova_node",
                  "concordance_df_nova_node",
                  "concordance_df_siata_node",
                  "concordance_df_hum_node",
                  "concordance_df_temp_node",
                  "concordance_nova_siata_node",
                  "concordance_nova_hum_node",
                  "concordance_nova_temp_node",
                  
                  "duplicates_node",
                  
                  "confi_df_node",
                  "confi_nova_node"]    
    dim_DQ= dim_node[cols].mean()
    dim_DQ.rename({'precision_df_node':          'precision_df_total', 
                   'precision_nova_node':        'precision_nova_total' , 
                   'uncertainty_node':           'uncertainty_total' , 
                   'accuracy_df_node':           'accuracy_df_total', 
                   'accuracy_nova_node':         'accuracy_nova_total', 
                   'completeness_df_node':       'completeness_df_total', 
                   'completeness_nova_node':     'completeness_nova_total', 
                   'concordance_df_nova_node':   'concordance_df_nova_total', 
                   'concordance_df_siata_node':  'concordance_df_siata_total', 
                   'concordance_df_hum_node':    'concordance_df_hum_total', 
                   'concordance_df_temp_node':   'concordance_df_temp_total', 
                   'concordance_nova_siata_node':'concordance_nova_siata_total', 
                   'concordance_nova_hum_node':  'concordance_nova_hum_total', 
                   'concordance_nova_temp_node': 'concordance_nova_temp_total', 
                   'duplicates_node':            'duplicates_total', 
                   'confi_df_node':              'confi_df_total', 
                   'confi_nova_node':            'confi_nova_total'        }, axis=1, inplace=True)
    
print("End Time: ", datetime.fromtimestamp(time.time()))
t1 = time.time() - t0
print("Elapsed Time: ", t1, " Seconds, or ",t1/60," Minutes")
#print(dim_node)



Start time:  2021-10-05 16:23:14.959601
Number of avaliable CPUs:  6
End Time:  2021-10-05 16:26:20.082072
Elapsed Time:  185.12612056732178  Seconds, or  3.0854353427886965  Minutes


DQ_INDEX by a weighted average function. The weights come from the Pair-Wise Comparison Matrix

In [5]:
dim_node["DQ_INDEX_NODE"]=  Wprecision*dim_node.loc[:,["precision_df_node","precision_nova_node"]].mean(axis=1,skipna=False)+\
                            Waccuracy*dim_node.loc[:,["accuracy_df_node","accuracy_nova_node"]].mean(axis=1,skipna=False)+\
                            Wcompleteness*dim_node.loc[:,["completeness_df_node","completeness_nova_node"]].mean(axis=1,skipna=False)+\
                            Wconfidence*dim_node.loc[:,["confi_df_node","confi_nova_node"]].mean(axis=1,skipna=False)+\
                            Wconcordance*dim_node.loc[:,["concordance_df_nova_node","concordance_df_siata_node","concordance_nova_siata_node"]].mean(axis=1,skipna=False)+\
                            Wdata_Redundancy*dim_node.loc[:,["duplicates_node"]].mean(axis=1,skipna=False)

#dim_node["precision_node"]= Wprecision*dim_node.loc[:,["precision_df_node","precision_nova_node"]].mean(axis=1)
#dim_node["accuracy_node"]= Waccuracy*dim_node.loc[:,["accuracy_df_node","accuracy_nova_node"]].mean(axis=1)
#dim_node["completeness_node"]= Wcompleteness*dim_node.loc[:,["completeness_df_node","completeness_nova_node"]].mean(axis=1)
#dim_node["confidence_node"]= Wconfidence*dim_node.loc[:,["confi_df_node","confi_nova_node"]].mean(axis=1)
#dim_node["concordance_node"]= Wconcordance*dim_node.loc[:,["concordance_df_nova_node","concordance_df_siata_node","concordance_nova_siata_node"]].mean(axis=1)
#dim_node["redundancy_node"]= Wdata_Redundancy*dim_node.loc[:,["duplicates_node"]].mean(axis=1)
#dim_node["DQ_INDEX_NODE"]=dim_node[["precision_node","accuracy_node","completeness_node","confidence_node","concordance_node","redundancy_node"]].sum(axis=1)

dim_DQ["DQ_INDEX_TOTAL"]=   Wprecision*dim_DQ[["precision_df_total","precision_nova_total"]].mean()+ \
                            Waccuracy*dim_DQ[["accuracy_df_total","accuracy_nova_total"]].mean()+ \
                            Wcompleteness*dim_DQ[["completeness_df_total","completeness_nova_total"]].mean()+ \
                            Wconfidence*dim_DQ[["confi_df_total","confi_nova_total"]].mean()+ \
                            Wconcordance*dim_DQ[["concordance_df_nova_total","concordance_df_siata_total","concordance_nova_siata_total"]].mean()+ \
                            Wdata_Redundancy*dim_DQ[["duplicates_total"]].mean()
#dim_node.head(6)
#dim_node.loc[dim_node.codigoSerial==49,]

Export to Google Sheets

In [6]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8'
#SAMPLE_RANGE_NAME = 'Class Data!A2:E'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    
    dim_time.replace(np.nan, '', inplace=True)
    dim_time['fechaHora'] = dim_time['fechaHora'].astype(str)
    #dim_time.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    dim_time.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    
    dim_node.replace(np.nan, '', inplace=True)
    #dim_time.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    dim_node.sort_values(by=['codigoSerial'],ignore_index=True)
    
    dim_DQ.replace(np.nan, '', inplace=True)
    #dim_time.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
        
    
    print("Clearing the DQ_TIME sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    result = sheet.values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        range='DQ_TIME!A1:Z1000000',
        
    ).execute()
    
    print("Clearing the DQ_NODE sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    result = sheet.values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        range='DQ_NODE!A1:Z1000000',
        
    ).execute()
    
    print("Clearing the DQ_TOTAL sheet in the Google Spreadsheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    result = sheet.values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        range='DQ_TOTAL!A1:Z1000000',
        
    ).execute()
    
    print("Exporting Data to Google Sheets")
    sheet = service.spreadsheets()
    result = sheet.values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_TIME!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_time.T.reset_index().T.values.tolist())
    ).execute()
    
    sheet = service.spreadsheets()
    result = sheet.values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_NODE!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_node.T.reset_index().T.values.tolist())
    ).execute()
    
    sheet = service.spreadsheets()
    result = sheet.values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_TOTAL!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_DQ.T.reset_index().T.values.tolist())
    ).execute()
    
    print("Exporting Data Finished")
#
if __name__ == '__main__':
    main()
    

Clearing the DQ_TIME sheet in the Google Spreadsheet
Clearing the DQ_NODE sheet in the Google Spreadsheet
Clearing the DQ_TOTAL sheet in the Google Spreadsheet
Exporting Data to Google Sheets
Exporting Data Finished


To Open the Spreadsheet

In [7]:
webbrowser.open('https://docs.google.com/spreadsheets/d/1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8/edit?usp=sharing')

True

To Open the Report

In [8]:
webbrowser.open('https://datastudio.google.com/s/ietWLq_iL-8')

True

In [ ]:
T1="2020-02-03 23:59:59"
T2="2020-02-03 00:00:00"
from datetime import datetime, timedelta


print(pd.Timestamp(T1)>=pd.Timestamp(T2).floor('1D') and pd.Timestamp(T1)<(pd.Timestamp(T2)+timedelta(minutes=1)).ceil('1D'))
print("Range: [",pd.Timestamp(T2).floor('1D'),",", (pd.Timestamp(T2)+timedelta(minutes=1)).ceil('1D'),"]")

In [ ]:
#arguments=[nodes=10, CC=CC, SS=CC, Distances=Distances, start_time=start_time, end_time=end_time]
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import uniform
from scipy.stats import norm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
import csv
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import seaborn as sn
import requests
import json
import haversine as hs
import wx
#hola

#FUNCTION TO READ THE PATH WITH A DIALOG BOX
def get_path(wildcard, title):
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST
    dialog = wx.FileDialog(None, title, wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPath()
    else:
        path = None
    dialog.Destroy()
    return path

#FUNCTION TO CLEAN AND SORT THE READ DATA.
def clean_sort_data(df_CC, df_SS):
    
    #ClEAN AND SORT THE CITIZEN SCIENCE DATASET
    #Remove outliers that are out of range, from documentation both nova and df range of measurements are [0,999], [-40,70], [1,100]
    #For PM2.5, Temperature and Relative Humidity Respectively.
    df_CC=df_CC.copy()
    df_CC.loc[df_CC["pm25_nova"]>999,"pm25_nova"]=np.nan
    df_CC.loc[df_CC["pm25_nova"]<0,"pm25_nova"]=np.nan
    df_CC.loc[df_CC["pm25_df"]>999,"pm25_df"]=np.nan
    df_CC.loc[df_CC["pm25_df"]<0,"pm25_df"]=np.nan
    df_CC.loc[df_CC["temperatura"]>70,"temperatura"]=np.nan
    df_CC.loc[df_CC["temperatura"]<-40,"temperatura"]=np.nan
    df_CC.loc[df_CC["humedad_relativa"]>100,"humedad_relativa"]=np.nan
    df_CC.loc[df_CC["humedad_relativa"]<1,"humedad_relativa"]=np.nan

    
    #Remove data above the whiskers of the boxplot: i.e. anomaly data
    Q1 = df_CC['pm25_df'].quantile(0.25)
    Q3 = df_CC['pm25_df'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_CC.loc[df_CC["pm25_df"]>=Q3 + 1.5 *IQR,"pm25_df"]=np.nan
    
    Q1 = df_CC['pm25_nova'].quantile(0.25)
    Q3 = df_CC['pm25_nova'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_CC.loc[df_CC["pm25_nova"]>=Q3 + 1.5 *IQR,"pm25_nova"]=np.nan
    
    Q1 = df_CC['temperatura'].quantile(0.25)
    Q3 = df_CC['temperatura'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_CC.loc[df_CC["temperatura"]>=Q3 + 1.5 *IQR,"temperatura"]=np.nan
    
    Q1 = df_CC['humedad_relativa'].quantile(0.25)
    Q3 = df_CC['humedad_relativa'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_CC.loc[df_CC["humedad_relativa"]>=Q3 + 1.5 *IQR,"humedad_relativa"]=np.nan
    
    #CREATE A DICTINARY CONTAINING THE DATASETS PER NODE
    grouped=df_CC.groupby(df_CC.codigoSerial)
    CC={}
    print("Citizen Scientist: ", sorted(list(df_CC.codigoSerial.unique())))
    for i in df_CC.codigoSerial.unique():
        CC[i] = grouped.get_group(i).sort_values(by=['fechaHora'],ignore_index=True)
    
    #ClEAN AND SORT THE SIATA STATIONS DATASET
    #Remove outliers that are out of range, from documentation both nova and df range of measurements are [0,999]
    df_SS=df_SS.copy()
    df_SS.loc[df_SS["pm25"]>999,"pm25"]=np.nan
    df_SS.loc[df_SS["pm25"]<0,"pm25"]=np.nan

    
    #Remove data above the whiskers of the boxplot: i.e. anomaly data
    Q1 = df_SS['pm25'].quantile(0.25)
    Q3 = df_SS['pm25'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_SS.loc[df_SS["pm25"]>=Q3 + 1.5 *IQR,"pm25"]=np.nan
    
    #CREATE A DICTINARY CONTAINING THE DATASETS PER SIATA STATION
    grouped=df_SS.groupby(df_SS.codigoSerial)
    SS={}
    print("Siata Stations: ", list(df_SS.codigoSerial.unique()))
    for j in df_SS.codigoSerial.unique():
        SS[j] = grouped.get_group(j).sort_values(by=['Fecha_Hora'],ignore_index=True)
    
    del grouped
    return CC, SS




#Precision
def precision(window):

    prec_df=1-(window['pm25_df'].std()/window['pm25_df'].mean())# 1-Coefficient of Variation (std/mean)
    prec_nova=1-(window['pm25_nova'].std()/window['pm25_nova'].mean())# 1-Coefficient of Variation (std/mean)
    prec_dict_time={"precision_df_time":prec_df,"precision_nova_time":prec_nova}
    return prec_dict_time


#Uncertainty
def uncertainty(window):
    uncert=1-np.sqrt((window.pm25_df-window.pm25_nova).pow(2).mean()/2)/((window.pm25_df+window.pm25_nova).mean()/2)
    uncer_dict_time={"uncertainty_time":uncert}
    return uncer_dict_time   


#ACCURACY
def accuracy(node,hour, window, Distances, SS):
    
    pm25_df_ave=window['pm25_df'].mean()
    pm25_nova_ave=window['pm25_nova'].mean()
    #print(pm25_df_ave, pm25_nova_ave)
    Closest_Station=Distances.codigoSerial_ES.loc[node]    
    Closest_Station2=Distances.codigoSerial_ES.loc[node] 
    #print(pm25_df_ave,pm25_nova_ave,Closest_Station,Closest_Station2)
    #print(Closest_Station in SS.keys())
    #print(hour in SS[Closest_Station].Fecha_Hora.values, hour)
   # print(SS[Closest_Station].Fecha_Hora)
    
    if (Closest_Station in SS.keys()) and (hour in SS[Closest_Station].Fecha_Hora.values):
        
        #print(SS[Closest_Station])
        #df_window.loc[df_window["fechaHora"]==ts,'pm25_nova_ave']
        v=SS[Closest_Station].loc[SS[Closest_Station].Fecha_Hora==hour,"pm25"].values[0]
        #print()
        #print(pm25_df_ave)
        accur_df=  max(0,1-abs(pm25_df_ave-v)/v)
        accur_nova= max(0,1-abs(pm25_nova_ave-v)/v)
    
    elif (Closest_Station2 in SS.keys()) and  (hour in SS[Closest_Station2].Fecha_Hora.values):
        
        v=SS[Closest_Station2].loc[SS[Closest_Station2].Fecha_Hora==hour,"pm25"].values[0]
        #v=SS[Closest_Station2].loc[hour,"pm25"]
        accur_df=  max(0,1-abs(pm25_df_ave-v)/v)
        accur_nova= max(0,1-abs(pm25_nova_ave-v)/v)
    else:
        accur_df=np.nan
        accur_nova=np.nan
    accu_dict_time={'accuracy_df_time':accur_df, 'accuracy_nova_time':accur_nova}
    return accu_dict_time


#CONCORDANCE
def concordance(node, hour, window, Distances, SS):
    Closest_Station=Distances.codigoSerial_ES.loc[node]
    Closest_Station2=Distances.codigoSerial_ES.loc[node]
    window=window.copy()
    if (Closest_Station in SS.keys()) and (hour in SS[Closest_Station].Fecha_Hora.values):
        
   
        v=SS[Closest_Station].loc[SS[Closest_Station].Fecha_Hora==hour,"pm25"].values[0]
        vm=window.pm25_df.mean()
        pair=[vm,v]
        window.loc[:,"v_pm25"]=v
        
        corr_df =   window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[0].abs()
        corr_nova = window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[1].abs()
        concordance_df_nova=corr_df.pm25_nova
        concordance_df_siata=corr_df.v_pm25
        concordance_df_hum=corr_df.humedad_relativa
        concordance_df_temp=corr_df.temperatura
        concordance_nova_siata=corr_nova.v_pm25
        concordance_nova_hum=corr_nova.humedad_relativa
        concordance_nova_temp=corr_nova.temperatura
   
            
    elif (Closest_Station2 in SS.keys()) and (hour in SS[Closest_Station2].Fecha_Hora.values):
        
        v=SS[Closest_Station2].loc[SS[Closest_Station2].Fecha_Hora==hour,"pm25"].values[0]
        vm=window.pm25_df.mean()
        pair=[vm,v]
        window.loc[:,"v_pm25"]=v
        
        corr_df =   window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[0].abs()
        corr_nova = window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[1].abs()
        concordance_df_nova=corr_df.pm25_nova
        concordance_df_siata=corr_df.v_pm25
        concordance_df_hum=corr_df.humedad_relativa
        concordance_df_temp=corr_df.temperatura
        concordance_nova_siata=corr_nova.v_pm25
        concordance_nova_hum=corr_nova.humedad_relativa
        concordance_nova_temp=corr_nova.temperatura
    else:
        concordance_df_nova=np.nan
        concordance_df_siata=np.nan
        concordance_df_hum=np.nan
        concordance_df_temp=np.nan
        concordance_nova_siata=np.nan
        concordance_nova_hum=np.nan
        concordance_nova_temp=np.nan
        
        pair=[np.nan,np.nan]
       
    conco_dict={"concordance_df_nova_time":concordance_df_nova,
                "concordance_df_siata":concordance_df_siata,
                "concordance_df_hum_time":concordance_df_hum,
                "concordance_df_temp_time":concordance_df_temp,
                "concordance_nova_siata":concordance_nova_siata,
                "concordance_nova_hum_time":concordance_nova_hum,
                "concordance_nova_temp_time":concordance_nova_temp,
                "vm":pair[0],
                "v":pair[1]}
    return conco_dict


#COMPLETENESS
def completeness(node, window,start_time, end_time):
    
    if window.fechaHora.min()==start_time and window.fechaHora.max()==end_time:
        ref_date_range = pd.DataFrame(pd.date_range(start_time,end_time, freq='1Min'),columns=["ref_fechaHora"])
    
    elif window.fechaHora.min()==start_time:
        ref_date_range = pd.DataFrame(pd.date_range(start_time, pd.Timestamp(start_time).ceil('60min')-timedelta(minutes=1), freq='1Min'),columns=["ref_fechaHora"])
        
    elif window.fechaHora.max()==end_time:
        ref_date_range = pd.DataFrame(pd.date_range(pd.Timestamp(end_time).floor('60min')-timedelta(minutes=1),pd.Timestamp(end_time), freq='1Min'),columns=["ref_fechaHora"])
    
    else:
        ref_date_range = pd.DataFrame(pd.date_range(window.fechaHora.min().floor('60min'),window.fechaHora.min().floor('60min')+timedelta(minutes=59), freq='1Min'),columns=["ref_fechaHora"])
        
    #Check for any missing date
    missing_dates = ref_date_range.loc[~ref_date_range.ref_fechaHora.isin(window.fechaHora),"ref_fechaHora"]
    
    #Add missing date rows
    for missing in missing_dates:
        window=window.append({"codigoSerial":node,"fechaHora":missing}, ignore_index = True)
    #Check for missing data
    missing_data_df=np.count_nonzero(np.isnan(window['pm25_df']))
    missing_data_nova=np.count_nonzero(np.isnan(window['pm25_nova']))
    comp_df=(1-missing_data_df/np.size(window.pm25_df))
    comp_nova=(1-missing_data_nova/np.size(window.pm25_nova))
    comp_dict_time={'completeness_df_time':comp_df, 'completeness_nova_time':comp_nova}        
    return comp_dict_time

def duplicates(window):
    repeated=len(window.index)-window['fechaHora'].nunique()
    duplic=1-repeated/len(window.index)
    dupli_dict_time={'duplicates_time':duplic} 
    return dupli_dict_time

def eval_dq(arguments):
    nodes=arguments[0]
    CC=arguments[1]
    SS=arguments[2]
    Distances=arguments[3]
    start_time=arguments[4]
    end_time=arguments[5]
    
    #1. For each citizen science (CC) node, get the groups (HOURLY GROUPS).
    #node_dataset=CC[nodes]
    node_dataset=CC[nodes][(CC[nodes]['fechaHora'] >= start_time) & (CC[nodes]['fechaHora'] <= end_time)]
    #print(node_dataset)
    #times = pd.to_datetime(node_dataset.fechaHora)
    hourly_groups=node_dataset.groupby([node_dataset.fechaHora.dt.floor('60min')])#Para agrupar por cada hora
    #hourly_groups.groups.keys()# O grupos.groups para obtener las claves de cada grupo, es decir cada hora
    del node_dataset
    

    dim_time = pd.DataFrame(
        columns =["codigoSerial",
                  "fechaHora",
                  "precision_df_time",
                  "precision_nova_time",
                  "uncertainty_time",
                  "accuracy_df_time",
                  "accuracy_nova_time",
                  "completeness_df_time",
                  "completeness_nova_time",
                  
                  "concordance_df_nova_time",
                  
                  "concordance_df_siata",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_df_hum_time",
                  "concordance_df_temp_time",
                  
                  "concordance_nova_siata",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_nova_hum_time",
                  "concordance_nova_temp_time",
                  "vm",
                  "v",
                 
                  "duplicates_time"])
    
    #2. For each group (hour in a CC node data), calculate the Dimension's DQ. (The functions should be applied to each group instead)
    for hour in hourly_groups.groups.keys():
        window=hourly_groups.get_group(hour)
        
        
        preci_dict_time=precision(window)
        uncer_dict_time=uncertainty(window)
        accur_dict_time=accuracy(nodes, hour, window, Distances, SS)
        conco_dict_time=concordance(nodes, hour, window, Distances, SS)
        comp_dict_time=completeness(nodes, window,start_time, end_time)
        dupli_dict_time=duplicates(window)
        
        DQ_dict_time={"codigoSerial":nodes,"fechaHora":pd.Timestamp(hour)}
        DQ_dict_time.update(preci_dict_time)
        DQ_dict_time.update(uncer_dict_time)
        DQ_dict_time.update(accur_dict_time)
        DQ_dict_time.update(conco_dict_time)
        DQ_dict_time.update(comp_dict_time)
        DQ_dict_time.update(dupli_dict_time)
        
        
        #3. Save the result file in the form: Node, Group (hour), DQ_1, DQ_2, DQ3, ... , DQIndex  (This is new)
        dim_time=dim_time.append(DQ_dict_time, ignore_index = True)
    
    dim_time['fechaHora']= pd.to_datetime(dim_time['fechaHora'])
    daily_groups=dim_time.groupby([dim_time.fechaHora.dt.floor('1D')])#Para agrupar por cada día
    print("here we go")
    #hola
    for day in daily_groups.groups.keys():
        day_window=daily_groups.get_group(day)
        #dim_time.loc[dim_time.fechaHora<=day,"concordance_df_siata"]=day_window.v.corr(day_window.vm)
        print((dim_time.fechaHora>=day.floor('1D'))*(dim_time.fechaHora<(day+timedelta(minutes=1)).ceil('1D')))
        dim_time.loc[(dim_time.fechaHora>=day.floor('1D'))&(dim_time.fechaHora<(day+timedelta(minutes=1)).ceil('1D')),"concordance_df_nova_time"]=1

        #day_window.v.corr(day_window.vm)
    
    return dim_time



dim_time=eval_dq([100,CC,SS,Distances,start_time,end_time])
dim_time[["codigoSerial","fechaHora","concordance_df_siata"]]

In [ ]:
import numpy as np
from scipy import stats


print(z_crit) # 1.959963984540054

In [ ]:
import pandas as pd
import numpy as np
df=pd.DataFrame([1,2,np.nan,4,np.nan])
n=np.count_nonzero(~np.isnan(df))
df.std().values[0]